In [155]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch 
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torchvision import transforms as tfs
from PIL import Image

In [156]:
data = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
y, x= data.iloc[:, 0], data.iloc[:, 1:]

test = torch.from_numpy(np.array(test).astype(np.float32))

x, y = np.array(x), np.array(y)
x_copy = x
y_copy = y
x, x_vl, y, y_vl = train_test_split(x, y, test_size = 0.3, shuffle = True)

x, y, x_vl, y_vl = torch.from_numpy(x.astype(np.float32)), torch.from_numpy(y.astype(np.float32)), torch.from_numpy(x_vl.astype(np.float32)), torch.from_numpy(y_vl.astype(np.float32))
x = x.view(-1, 1, 28, 28)
x_vl = x_vl.view(-1, 1, 28, 28)
test = test.view(-1, 1, 28, 28)
print(x.shape, x_vl.shape, test.shape, y.shape)
x_copy.shape

torch.Size([29400, 1, 28, 28]) torch.Size([12600, 1, 28, 28]) torch.Size([28000, 1, 28, 28]) torch.Size([29400])


(42000, 784)

In [157]:
im_aug = tfs.Compose([
    tfs.Resize(32),
    tfs.RandomHorizontalFlip(),
    tfs.RandomCrop(28),
    tfs.ColorJitter(contrast=1),
    tfs.ToTensor(),
])

In [158]:
x_copy = x_copy.reshape(-1, 28, 28)
x_aug = torch.tensor(x_copy)
for idx, _ in enumerate(x_copy):
    x_aug[idx] = im_aug(Image.fromarray(np.uint8(x_copy[idx])))

In [159]:
x_aug = x_aug.reshape(-1, 1, 28, 28)
print(x.shape, x_aug.shape)
x = torch.cat((x, x_aug), 0)
x = x.float()
print(x.shape)

torch.Size([29400, 1, 28, 28]) torch.Size([42000, 1, 28, 28])
torch.Size([71400, 1, 28, 28])


In [160]:
y_aug = torch.tensor(y_copy)
y = torch.cat((y, y_aug))
x = x.float()
y = y.long()
print(x.shape, y.shape)

torch.Size([71400, 1, 28, 28]) torch.Size([71400])


In [161]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv = nn.Sequential( 
            # size: 28*28
            nn.Conv2d(1,8,3,1,1), # in_channels out_channels kernel_size stride padding
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8,16,3,1,1), 
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # size: 14*14
            nn.Conv2d(16,16,3,1,1), 
            nn.ReLU(),
            nn.Conv2d(16,8,3,1,1), 
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Sequential(
            # size: 7*7
            nn.Linear(8*7*7,256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256,10)
        )

    
    def forward(self, img):
        x = self.conv(img)
        o = self.fc(x.view(x.shape[0],-1))
        return o


In [162]:
bs = 64
trainDS = TensorDataset(x, y)
testDS = TensorDataset(test)
validDS = TensorDataset(x_vl, y_vl)

trainDL = DataLoader(trainDS, batch_size = bs, shuffle = True)
testDL = DataLoader(testDS, batch_size = bs, shuffle = True)
validDL = DataLoader(validDS, batch_size = bs, shuffle = True)

In [163]:
criterion = nn.CrossEntropyLoss()
net = Net()
optim = torch.optim.SGD(net.parameters(), lr = 0.01)
epochs = 22

In [ ]:
for epoch in range(epochs):
    epoc_train_loss = 0.0
    epoc_train_corr = 0.0
    epoc_valid_corr = 0.0
    print('Epoch:{}/{}'.format(epoch,epochs))
    
    for data in trainDL:
        
        images,labels = data
        outputs = net(images)               
        optim.zero_grad()
        loss = criterion(outputs,labels.long())
        loss.backward()
        optim.step()
        
        epoc_train_loss += loss.item()
        outputs = torch.max(outputs,1)[1]
        epoc_train_corr += torch.sum(outputs==labels)
    
    with torch.no_grad():
        for data in validDL:

            images,labels = data
            labels = labels


            outputs = net(images)
            outputs = torch.max(outputs.data,1)[1]

            epoc_valid_corr += torch.sum(outputs==labels.data)
    
    
    print("loss is :{:.4f},Train Accuracy is:{:.4f}%,Test Accuracy is:{:.4f}%".format(epoc_train_loss/len(trainDS),100*epoc_train_corr/len(trainDS),100*epoc_valid_corr/len(validDS)))

Epoch:0/22
loss is :0.0322,Train Accuracy is:22.2367%,Test Accuracy is:76.3968%
Epoch:1/22
loss is :0.0247,Train Accuracy is:40.5308%,Test Accuracy is:86.8730%
Epoch:2/22
loss is :0.0235,Train Accuracy is:43.8599%,Test Accuracy is:89.6190%
Epoch:3/22
loss is :0.0226,Train Accuracy is:48.3725%,Test Accuracy is:88.6905%
Epoch:4/22
loss is :0.0210,Train Accuracy is:52.7479%,Test Accuracy is:87.9444%
Epoch:5/22
loss is :0.0194,Train Accuracy is:56.2423%,Test Accuracy is:88.8016%
Epoch:6/22
loss is :0.0180,Train Accuracy is:58.5868%,Test Accuracy is:91.2619%
Epoch:7/22
loss is :0.0170,Train Accuracy is:60.4888%,Test Accuracy is:90.5873%
Epoch:8/22
loss is :0.0164,Train Accuracy is:61.5182%,Test Accuracy is:91.0159%
Epoch:9/22
loss is :0.0160,Train Accuracy is:62.6443%,Test Accuracy is:91.2778%
Epoch:10/22
loss is :0.0156,Train Accuracy is:63.3193%,Test Accuracy is:92.1825%
Epoch:11/22
loss is :0.0154,Train Accuracy is:63.6471%,Test Accuracy is:93.0079%
Epoch:12/22
loss is :0.0150,Train Accu

In [ ]:
y_pred = net(test)
y_pred = torch.max(y_pred, 1)[1]
print(y_pred.shape)

In [ ]:
out = np.array(y_pred)
out = {'ImageId': range(1, len(out) + 1), 'Label': out}
out = pd.DataFrame(out)
print(out.shape)
out.to_csv('submission.csv', index = False)